In [15]:
using Flux
using CSV

# Define a function to load and preprocess the data for the first 10 letters
function load_data(file_path)
    data = CSV.File(file_path)
    
    # Filter data for the first 10 letters (A to J)
    data = filter(row -> Char(row[1]) in 'A':'J', data)
    
    labels = [Char(row[1]) for row in data]
    pixels = [Float32.(row[2:end]) for row in data]
    
    x = hcat(pixels...) # Convert to a matrix
    y = Flux.onehotbatch(labels, 'A':'J')
    
    return (x, y)
end
# Load the data
x_train, y_train = load_data("C:/Users/Jash/Documents/Research/Alphabet/handwritten_data_785.csv")

# Define the CNN model
model = Chain(
    Conv((3, 3), 1=>16, relu),
    MaxPool((2, 2)),
    Conv((3, 3), 16=>32, relu),
    MaxPool((2, 2)),
    x -> reshape(x, :, size(x, 4)),
    Dense(32 * 5 * 5, 128, relu),
    Dense(128, 10)  # Output layer with 10 units
)


# Define the loss function
loss(x, y) = Flux.crossentropy(model(x), y)

# Create an optimizer
optimizer = ADAM(0.001)

# Training loop
n_epochs = 10
batch_size = 64

for epoch in 1:n_epochs
    for i in 1:batch_size:size(x_train, 2)-batch_size+1
        x_batch = x_train[:, i:i+batch_size-1]
        y_batch = y_train[:, i:i+batch_size-1]
        gradient = Flux.gradient(() -> loss(x_batch, y_batch), params(model))
        Flux.update!(optimizer, params(model), gradient)
    end
    
    # Create a batch of 28x28x1 images as sample input
    num_samples = 10  # Adjust the batch size as needed
    sample_input = rand(Float32, 28, 28, 1, num_samples)  # 1 for the number of channels (grayscale)

    output = model(sample_input)
    println("Epoch $epoch - Output layer for sample input:\n", output)
end

Epoch 1 - Output layer for sample input:


Float32[0.07009542 0.097594485 0.049304157 0.099986516 0.10720212 0.1080036 0.08888322 0.1181228 0.11341427 0.066077515; 0.03549628 0.055210248 0.00066090375 0.01951421 0.051332682 0.041326568 -0.023401298 0.042106166 0.03768254 0.040879652; 0.008340538 0.027666047 -0.019799992 0.017396405 0.019272447 -0.01668033 0.035260215 0.06810349 -0.020892957 0.022847164; -0.12049301 -0.12268784 -0.18451506 -0.1960853 -0.18399197 -0.13936001 -0.21713802 -0.18259585 -0.14987946 -0.13116422; 0.11884515 0.13187984 0.16609848 0.19393295 0.11111002 0.14905764 0.092268735 0.13160387 0.11089912 0.07638346; -0.17108724 -0.15503156 -0.20544162 -0.1961849 -0.2152022 -0.2552159 -0.17118357 -0.1910618 -0.21912143 -0.23884587; 0.04564716 -0.029379677 0.020311534 -0.006034352 0.046791982 0.0058927573 0.04648837 0.024995618 0.02012093 0.026695881; -0.080791235 -0.116359845 -0.14108735 -0.105164215 -0.11730066 -0.08626516 -0.12788498 -0.1305369 -0.115458414 -0.17405643; 0.08063979 0.04949028 0.016846912 0.083096

; -0.17033908 -0.17239794 -0.19436146 -0.17684615 -0.15860659 -0.14677618 -0.14735009 -0.17797302 -0.20095904 -0.13713825; 0.11971603 0.07134 0.11929874 0.12550136 0.080709696 0.10537202 0.11415338 0.158739 0.1257925 0.14633366; -0.2030344 -0.20522171 -0.23869544 -0.12524447 -0.20525712 -0.16877767 -0.15666533 -0.15543832 -0.19494098 -0.166497; 0.005629547 0.03050983 0.04318143 -0.04727347 0.03667624 0.018328626 -0.002126284 0.02796273 0.077369735 0.014357521; -0.113460094 -0.11963403 -0.12253465 -0.08480401 -0.16562419 -0.13918635 -0.13374165 -0.1373528 -0.1281962 -0.15437794; 0.068377204 0.0715793 0.011024124 0.05762861 0.057295416 0.013008514 0.048321594 0.043093033 0.0020465732 0.014583196; 0.030979237 0.012344192 0.020403579 0.03778798 0.02324425 -0.031862475 0.04493231 0.058190014 0.052204374 0.050361637]
Epoch 10 - Output layer for sample input:
Float32[0.029373437 0.11070284 0.11353538 0.10534716 0.108602166 0.10000484 0.13400775 0.05369602 0.106920384 0.032594766; 0.0862207 0.